---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    ''' Leitura de arquivo basica no python
    file = open("university_towns.txt",'r')
    print(file.read())'''
    
    # Leitura do arquivo .txt direto para DataFrame
    df = pd.read_csv("university_towns.txt",sep='\t', header=None)
    
    df = df[0]
    
    # Busca da substring '[edit]' presente em todos os estados
    sub = '[edit]'    
    df_states = (df[df.str.find(sub)>0]
                 .str.replace("\[.*","")
                 .str.strip())
    
    # Enfatizando que é dataframe
    df = df.to_frame()
    
    # Inicialização da key 'State'
    df['State'] = '0'
    
    # Loop para atribuição dos estados baseado na lista de indices do texto como um todo
    j = 0
    for i in df.index:
        if(j+1<len(df_states.index)):
            if ((df.index[i]>df_states.index[j])&(df.index[i]<df_states.index[j+1])):
                df.loc[i]['State'] = df_states[df_states.index[j]]
            if (df.index[i]>=df_states.index[j+1]):
                j += 1
                df.loc[i]['State'] = df_states[df_states.index[j]]
        else:
            df.loc[i]['State'] = df_states[df_states.index[j]]
    
    # Formatação dos nomes de cidade, foi utilizada uma expressão regex onde:
    # '\(' representa alguma instancia do caracter '('
    # '.' representa qualquer caracter
    # '*' este operador repete a busca anterior até o final da string
    # '\[' representa alguma instancia do caracter '('
    # .str.strip() serve para tirar espaços a direita e a esquerda
    df[0] = (df[0].str.replace("\(.*","")
             .str.replace("\[.*","")
             .str.strip())
    
    # Alterando o nome da coluna de regiões
    df = df.rename(columns = {0:'RegionName'})
    
    # Retirando os estados da lista principla e reiniciando os indices
    # A função drop recebeu df_states.index que é a lista de indices a serem retirados
    df = (df.drop(df_states.index)
          .reset_index(drop=True))
    
    return df[['State','RegionName']]
#print(repr(get_list_of_university_towns()))

In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    colnames = ['Quarter',
               'GDP_Current']
    
    # Leitura e formatação do database de GDP
    df = (pd.read_excel('gdplev.xls',
                       skiprows=5,
                       usecols=[i for i in range(4,6)],
                       headers=True,
                       names = colnames)
                       .drop({0,1}).reset_index(drop=True))
    
    year = '2000q1'
    df = df[df['Quarter'].values.tolist().index('2000q1'):].reset_index(drop=True)
    
    # Loop para encontrar o começo da recessão
    for i in df.index:
        if (i+2>len(df.index)):
            return None
        if ((df.loc[i]['GDP_Current']>df.loc[i+1]['GDP_Current'])&(df.loc[i+1]['GDP_Current']>df.loc[i+2]['GDP_Current'])):
            return df.loc[i]['Quarter']
#print(get_recession_start())

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    colnames = ['Quarter',
               'GDP_Current']
    
    # Leitura e formatação do database de GDP
    df = (pd.read_excel('gdplev.xls',
                       skiprows=5,
                       usecols=[i for i in range(4,6)],
                       headers=True,
                       names = colnames)
                       .drop({0,1}).reset_index(drop=True))
    year = '2000q1'
    df = df[df['Quarter'].values.tolist().index('2000q1'):].reset_index(drop=True)
    
    # Obtenção do começo da recessão
    reces_start = get_recession_start()
    
    # Controle para o caso de não ter recessão no periodo
    if(reces_start==None):
        return None
    else:
        df = df[df['Quarter'].values.tolist().index(reces_start):].reset_index(drop=True)
     
    # Loop para encontrar o começo da recessão
    for i in df.index:
        if (i+2>len(df.index)):
            return None
        if ((df.loc[i]['GDP_Current']<df.loc[i+1]['GDP_Current'])&(df.loc[i+1]['GDP_Current']<df.loc[i+2]['GDP_Current'])):
            return df.loc[i+2]['Quarter']
#print(get_recession_end())

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    colnames = ['Quarter',
               'GDP_Current']
    # Leitura e formatação do database de GDP
    df = (pd.read_excel('gdplev.xls',
                       skiprows=5,
                       usecols=[i for i in range(4,6)],
                       headers=True,
                       names = colnames)
                       .drop({0,1}).reset_index(drop=True))
    year = '2000q1'
    df = (df[df['Quarter'].values.tolist().index('2000q1'):]
          .reset_index(drop=True))
    
    # Obtenção do começo e fim da recessão
    reces_start = get_recession_start()
    reces_end = get_recession_end()
    
    # Controle para o caso de não ter recessão no periodo ou ela ainda não ter terminado
    if((reces_start==None)|(reces_end==None)):
        return None
    else:
        # Obtenção do range de dados para a pesquisa do menor  valor
        df = (df[df['Quarter'].values.tolist().index(reces_start):
                 df['Quarter'].values.tolist().index(reces_end)]
                 .reset_index(drop=True))
        
    # Obtenção do valor minimo de GDP e de seu indice que é passado no df com os quartis
    return df['Quarter'][df['GDP_Current'].idxmin()]
#print(get_recession_bottom())

In [7]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    # Fazer calculos das médias para cada quadrante
    sub = '2000-01'
    datas = df[df.columns[df.columns.get_loc(sub):]]
    datas.columns = pd.to_datetime(datas.columns)
    datas = datas.resample('1q',how='mean',axis=1)
    
    '''# Determinar os nomes de quadrantes para nomear as colunas(Alternativo)
    colnames = []
    for i in range(0,17):
        if(i<10):
            for j in range(1,5):
                colnames.append('200{}q{}'.format(i,j))
        else:
            for j in range(1,5):
                colnames.append('20{}q{}'.format(i,j))
    colnames.pop(-1)'''
    
    # Renomeando as colunas de data
    
    # Isso funciona por que era do tipo datetime
    datas = datas.rename(columns=lambda col: '{}q{}'.format(col.year, col.quarter)) 
    
    # Subistituição de sigla para nome de estado e adicionando a coluna ao df
    datas['State'] = df['State'].replace(states)
    
    # Adicionando o campo RegionName ao df
    datas['RegionName'] = df['RegionName']
    
    # Preparando os multiindices
    datas = datas.set_index(['State','RegionName'])
    return  datas
#print(convert_housing_data_to_quarters())

In [78]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # Obtenção da base de dados
    uni_town = get_list_of_university_towns()
    housing = convert_housing_data_to_quarters()
    reces_start = get_recession_start()
    reces_end = get_recession_end()
    reces_bottom = get_recession_bottom()
    
    # Obtenção do range de quartis para a estimativa
    housing = housing[housing.columns[housing.columns.get_loc(reces_start)-1:
                                      housing.columns.get_loc(reces_end)]]
    
    # Obtenção dos dataframes para cidades com universidade e sem universidade
    uni_housing = pd.merge( uni_town, housing, how='inner', left_on=['State','RegionName'], right_index=True).reset_index(drop=True)
    non_uni_housing = pd.merge( uni_town, housing, how='outer', left_on=['State','RegionName'], right_index=True).reset_index(drop=True)
    
    # Expreção para o t-test: price_ratio=quarter_before_recession/recession_bottom
    price_ratio_uni = (uni_housing[uni_housing.columns[uni_housing.columns.get_loc(reces_start)-1]]/
                       uni_housing[uni_housing.columns[uni_housing.columns.get_loc(reces_bottom)]])
    
    price_ratio_non_uni = (non_uni_housing[non_uni_housing.columns[non_uni_housing.columns.get_loc(reces_start)-1]]/
                           non_uni_housing[non_uni_housing.columns[non_uni_housing.columns.get_loc(reces_bottom)]])
    
    # Teste de t de student para verificar a hipotese
    t_test = ttest_ind(price_ratio_uni.dropna(),price_ratio_non_uni.dropna())
    
    if(t_test[1]>0.01):
        return (False, t_test[1],"non-university town")
    else:
        return (True, t_test[1], "university town")
#print(run_ttest())

(True, 0.0033861978483629026, 'university town')
